Siemanko
Hejka!!

1.

In [32]:
from sklearn.metrics import (accuracy_score, f1_score)
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.svm import SVC
from sklearn.utils import resample
import tensorflow as tf
from random import sample 

In [17]:
(x_train_f, y_train_f), (x_test_f, y_test_f) = tf.keras.datasets.fashion_mnist.load_data()

In [24]:
x_train = x_train_f.reshape((x_train_f.shape[0],-1))
x_test = x_test_f.reshape((x_test_f.shape[0],-1))
y_train = y_train_f.reshape((y_train_f.shape[0],))
y_test = y_test_f

2.

In [4]:
scaler = StandardScaler()

scaler.fit(x_train)

x_train_scaled = scaler.transform(x_train)
x_test_scaled = scaler.transform(x_test)

pca = PCA(n_components=50)
pca.fit(x_train_scaled)

x_train_pca = pca.transform(x_train_scaled)
x_test_pca = pca.transform(x_test_scaled)

Pełne dane

In [16]:
clf = SVC(probability=True)
clf.fit(x_train_pca, y_train)

SVC(C=1.0, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=True, random_state=None, shrinking=True, tol=0.001,
    verbose=False)

In [17]:
y_pred = clf.predict(x_test_pca)

In [22]:
pred = clf.predict_proba(x_test_pca)

NameError: name 'clf' is not defined

In [19]:
accuracy = accuracy_score(y_test, y_pred)
f_score = f1_score(y_test, y_pred, average='weighted')

In [20]:
print(accuracy)
print(f_score)

0.8666
0.8660852238643809


Niepełne dane i cechy

In [47]:
samples_all = x_train_pca.shape[0]
samples_all_test = x_test_pca.shape[0]
features_all = x_train_pca.shape[1]

In [48]:
n_samples = [0.1, 0.25, 0.35, 0.5]
n_features = [0.1, 0.2, 0.3]

In [49]:
x_train_10, y_train_10 = resample(x_train_pca, y_train, n_samples=7000, replace=False, random_state=0)
x_test_10, y_test_10 = resample(x_test_pca, y_test, n_samples=7000, replace=False, random_state=0)

In [50]:
def get_minor_classifiers(samples_perc, features_perc):
    feature_list = [n for n in range(50)]
    classifiers = []
    
    for samples in samples_perc:
        for features in features_perc:
            f = sample(feature_list, int(features_all * features))
            x_train_f = x_train_pca[:,f]
            
            x_train_s, y_train_s = resample(x_train_pca, y_train, n_samples=int(samples * samples_all), replace=False, random_state=0)
            x_test_s, y_test_s = resample(x_test_pca, y_test, n_samples=int(samples * samples_all_test), replace=False, random_state=0)
            
            svm_clf = SVC(probability=True)
            svm_clf.fit(x_train_s, y_train_s)
            
            y_pred = svm_clf.predict(x_test_pca)
            pred = svm_clf.predict_proba(x_test_pca)
            
            classifiers.append((y_pred, pred))
            
    return classifiers

In [ ]:
get_minor_classifiers(n_samples, n_features)

[9 2 1 ... 8 1 5]
[9 2 1 ... 8 1 5]
[9 2 1 ... 8 1 5]
[9 2 1 ... 8 1 5]
[9 2 1 ... 8 1 5]
[9 2 1 ... 8 1 5]
[9 2 1 ... 8 1 5]
[9 2 1 ... 8 1 5]
[9 2 1 ... 8 1 5]
[9 2 1 ... 8 1 5]
[9 2 1 ... 8 1 5]


In [ ]:
clf1 = SVC(probability=True)
clf1.fit(x_train_10, y_train_10)

In [8]:
y_pred = clf1.predict(x_test_10)

In [9]:
accuracy = accuracy_score(y_test_10, y_pred)
f_score = f1_score(y_test_10, y_pred, average='weighted')

In [10]:
print(accuracy)
print(f_score)

0.8362857142857143
0.8338516634129931


3a.

In [ ]:
clf_7iter = SVC(probability=True,max_iter=7)

In [ ]:
clf_7iter.fit(x_train, y_train)
y_pred_7ter = clf_7iter.predict(x_test)
pred_7iter = clf_7iter.predict_proba(x_test)

In [ ]:
pred_7iter

In [ ]:
from sklearn.metrics import accuracy_score
accuracy_7iter = accuracy_score(y_test, y_pred_7ter)
accuracy = accuracy_score(y_test, y_pred)

In [ ]:
accuracy_7iter

In [ ]:
accuracy